<a href="https://colab.research.google.com/github/sawsanzaky/computer-vision/blob/main/cv_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.google.com/presentation/d/1o1islRG7R89F_c3m_kDTpBQXiRAhGmYp/edit#slide=id.g101e7a8516b_0_9

In [8]:
#Training/Validation Images
!gdown --id  1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86  #FOR DOWNLOAD DATASET 


Downloading...
From: https://drive.google.com/uc?id=1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
To: /content/fairface-img-margin025-trainval.zip
100% 578M/578M [00:11<00:00, 50.0MB/s]


In [10]:
#Training Labels
!gdown --id 1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH


Downloading...
From: https://drive.google.com/uc?id=1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
To: /content/fairface_label_train.csv
100% 3.79M/3.79M [00:00<00:00, 58.6MB/s]


In [11]:
#Validation Labels
!gdown --id 1wOdja-ezstMEp81tX1a-EYkFebev4h7D

Downloading...
From: https://drive.google.com/uc?id=1wOdja-ezstMEp81tX1a-EYkFebev4h7D
To: /content/fairface_label_val.csv
100% 448k/448k [00:00<00:00, 58.2MB/s]


In [17]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [50]:
label_train = pd.read_csv("fairface_label_train.csv")
label_val = pd.read_csv("fairface_label_val.csv")

In [51]:
label_train.head()

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True


In [52]:
mapping_dictDayNight={'Female':1,'Male':0}
label_train['gender']=label_train['gender'].map(mapping_dictDayNight)

label_train.head()

,file,age,gender,race,service_test
0,train/1.jpg,50-59,0,East Asian,True
1,train/2.jpg,30-39,1,Indian,False
2,train/3.jpg,3-9,1,Black,False
3,train/4.jpg,20-29,1,Indian,True
4,train/5.jpg,20-29,1,Indian,True


In [53]:
service_test={True:1,False:0}
label_train['service_test']=label_train['service_test'].map(service_test)

label_train.head()

,file,age,gender,race,service_test
0,train/1.jpg,50-59,0,East Asian,1
1,train/2.jpg,30-39,1,Indian,0
2,train/3.jpg,3-9,1,Black,0
3,train/4.jpg,20-29,1,Indian,1
4,train/5.jpg,20-29,1,Indian,1


In [54]:
label_train.race.unique()

array(['East Asian', 'Indian', 'Black', 'White', 'Middle Eastern',
       'Latino_Hispanic', 'Southeast Asian'], dtype=object)

In [55]:
race={'East Asian':0, 'Indian':1, 'Black':2, 'White':3, 'Middle Eastern':4,
       'Latino_Hispanic':5, 'Southeast Asian':6}
label_train['race']=label_train['race'].map(race)

label_train.sample(10)

,file,age,gender,race,service_test
39458,train/39459.jpg,40-49,0,3,0
79400,train/79401.jpg,20-29,1,5,0
62049,train/62050.jpg,10-19,1,1,1
15876,train/15877.jpg,30-39,0,3,0
68713,train/68714.jpg,10-19,1,4,1
11392,train/11393.jpg,3-9,1,6,0
67926,train/67927.jpg,30-39,1,3,0
58133,train/58134.jpg,20-29,0,1,1
29526,train/29527.jpg,40-49,0,6,0
59619,train/59620.jpg,10-19,1,6,0


In [56]:
label_train.sample(10)

,file,age,gender,race,service_test
68729,train/68730.jpg,3-9,1,6,1
67860,train/67861.jpg,20-29,1,3,0
35244,train/35245.jpg,30-39,1,0,1
72292,train/72293.jpg,40-49,0,3,1
70852,train/70853.jpg,60-69,1,6,0
53065,train/53066.jpg,0-2,1,3,0
20749,train/20750.jpg,30-39,1,5,1
29483,train/29484.jpg,30-39,0,0,1
84193,train/84194.jpg,10-19,0,1,1
22989,train/22990.jpg,30-39,0,5,0


In [12]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "./data/hymenoptera_data"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

Model Training and Validation Code

In [57]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history   #return the best evalute 

In [59]:
train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False)


NameError: ignored

Set Model Parameters’ .requires_grad attribute


In [14]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [16]:
(fc): Linear(in_features=512, out_features=1000, bias=True)


NameError: ignored

Load Data
